# Lendo os PDFs

In [39]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
import re
import faiss
import numpy as np
import pandas as pd

def load_pdf(path):
    reader = PdfReader(path)
    pages = []
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        pages.append({"page_number": i+1, "text": text})
    return pages

In [40]:
import spacy

try:
    nlp = spacy.load("pt_core_news_lg")
except OSError:
    raise RuntimeError("Modelo 'pt_core_news_lg' não encontrado. Instale com: python -m spacy download pt_core_news_lg")


def extract_entities_pt(text):
    doc_q = nlp(text)
    ents = [(ent.text, ent.label_) for ent in doc_q.ents]
    # Remove duplicatas mantendo ordem
    seen = set()
    unique_ents = []
    for t, l in ents:
        key = (t.lower(), l)
        if key not in seen:
            seen.add(key)
            unique_ents.append({"text": t, "label": l})
    return unique_ents

def expand_query_with_entities(query: str, entities: list):
    if not entities:
        return query
    # Cria um reforço leve adicionando entidades no final
    ents_text = " ".join([e["text"] for e in entities])
    expanded = f"{query} {ents_text}".strip()
    return expanded

In [41]:
def tratar_texto(text):
    if not text:
        return ""
    text = re.sub(r'\r\n|\r', '\n', text)
    text = re.sub(r'\n{2,}', '\n\n', text)
    text = re.sub(r'[ \t]{2,}', ' ', text)
    text = text.strip()
    return text

In [42]:
pdfs = [
    {
        "file": "C:\\Users\\Guilherme Monteiro\\Desktop\\codigos_gerais\\proj4\\textos\\Apologia-de-Socrates-de-Platao.pdf",
        "titulo": "Apologia de Sócrates",
        "autor": "Platão",
        "prefix": "apologia"
    },
    {
        "file": "C:\\Users\\Guilherme Monteiro\\Desktop\\codigos_gerais\\proj4\\textos\\Carta-a-Meneceu-Sobre-a-Felicidade-por-Epicuro.pdf",
        "titulo": "Carta a Meneceu",
        "autor": "Epicuro",
        "prefix": "meneceu"
    },
    {
        "file": "C:\\Users\\Guilherme Monteiro\\Desktop\\codigos_gerais\\proj4\\textos\\O-Mito-da-Caverna-trecho-do-A-Republica-de-Platao.pdf",
        "titulo": "O Mito da Caverna",
        "autor": "Platão",
        "prefix": "caverna"
    }
]

In [43]:
all_documents = []

for item in pdfs:
    pages = load_pdf(item["file"])
    for p in pages:
        all_documents.append({
            "titulo": item["titulo"],
            "autor": item["autor"],
            "prefix": item["prefix"],
            "pagina_pdf": p["page_number"],
            "texto_bruto": tratar_texto(p["text"])
        })

Ignoring wrong pointing object 5 0 (offset 0)


In [44]:
len(all_documents), all_documents[-1]

(37,
 {'titulo': 'O Mito da Caverna',
  'autor': 'Platão',
  'prefix': 'caverna',
  'pagina_pdf': 5,
  'texto_bruto': 'caso eis o que me aparece tal como me aparece; nos últimos limites \ndo mundo inteligível aparece-me a idéia do Bem, que se percebe com \ndificuldade, mas que não se pode ver sem concluir que ela é a causa \nde tudo o que há de reto e de belo. No mundo visível, ela gera a luz e \no senhor da luz, no mundo inteligível ela própria é a soberana que \ndispensa a verdade e a inteligência. Acrescento que é preciso vê-la se \nquer comportar-se com sabedoria, seja na vida privada, seja na vida \npública.\nGlauco: Tanto quanto sou capaz de compreender-te, concordo \ncontigo.\nReferência:\nA Alegoria da caverna: A Republica, 514a-517c tradução de Lucy \nMagalhães. \nIn: MARCONDES, Danilo. Textos Básicos de Filosofia: dos Pré-\nsocráticos a Wittgenstein. 2a ed. Rio de Janeiro: Jorge Zahar Editor, \n2000.'})

## Criando os chunks 

In [45]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", "? ", "! ", " "]
)

chunks_final = []

for doc in all_documents:
    splits = splitter.split_text(doc["texto_bruto"])
    
    for idx, chunk in enumerate(splits):
        chunk_id = f"{doc['prefix']}_{doc['pagina_pdf']}_{idx}"
        
        # Extrai entidades nomeadas (NER) com spaCy
        doc_spacy = nlp(chunk)
        ner = [{"text": ent.text, "label": ent.label_} for ent in doc_spacy.ents]
        
        chunks_final.append({
            "titulo": doc["titulo"],
            "autor": doc["autor"],
            "capitulo": None,
            "pagina_pdf": doc["pagina_pdf"],
            "chunk_id": chunk_id,
            "texto": chunk,
            "ner": ner
        })

len(chunks_final)

130

In [46]:
chunks_final[-1]

{'titulo': 'O Mito da Caverna',
 'autor': 'Platão',
 'capitulo': None,
 'pagina_pdf': 5,
 'chunk_id': 'caverna_5_0',
 'texto': 'caso eis o que me aparece tal como me aparece; nos últimos limites \ndo mundo inteligível aparece-me a idéia do Bem, que se percebe com \ndificuldade, mas que não se pode ver sem concluir que ela é a causa \nde tudo o que há de reto e de belo. No mundo visível, ela gera a luz e \no senhor da luz, no mundo inteligível ela própria é a soberana que \ndispensa a verdade e a inteligência. Acrescento que é preciso vê-la se \nquer comportar-se com sabedoria, seja na vida privada, seja na vida \npública.\nGlauco: Tanto quanto sou capaz de compreender-te, concordo \ncontigo.\nReferência:\nA Alegoria da caverna: A Republica, 514a-517c tradução de Lucy \nMagalhães. \nIn: MARCONDES, Danilo. Textos Básicos de Filosofia: dos Pré-\nsocráticos a Wittgenstein. 2a ed. Rio de Janeiro: Jorge Zahar Editor, \n2000.',
 'ner': [{'text': 'Bem', 'label': 'MISC'},
  {'text': 'Glauco', '

## Salvando cópia em JSON

In [47]:
output_file = "chunks_filosofia.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(chunks_final, f, ensure_ascii=False, indent=2)

output_file

'chunks_filosofia.json'

### Criando os Embeddings
Aqui, estou utilizando um modelo multi linguas do Alibaba, treinado para criação de embeddings semanticos

In [48]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# BERTimbau (Portuguese BERT)
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert_model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert_model.eval()


def encode_text_mean_pool(text: str):
    """Encode a single text using BERTimbau and return a mean-pooled embedding.
    Excludes [CLS]/[SEP] special tokens from the mean.
    """
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        token_embeds = outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden]
    # Identify special tokens and mask them
    ids = inputs['input_ids'].squeeze(0)
    # Exclude first ([CLS]) and last ([SEP]) positions when possible
    start = 1 if ids.numel() > 2 else 0
    end = -1 if ids.numel() > 2 else ids.numel()
    valid_tokens = token_embeds[start:end]
    if valid_tokens.numel() == 0:
        valid_tokens = token_embeds
    vec = valid_tokens.mean(dim=0)  # [hidden]
    return vec.cpu().numpy()


lista_emb = []
texts = []
metadados = []

for chunk in chunks_final:
    # concatena metadados ao texto para melhorar recuperação
    ner_text = " ".join([e.get("text", "") for e in chunk.get("ner", [])])
    texto_meta = (
        f"TITULO: {chunk['titulo']} AUTOR: {chunk['autor']} PAGINA: {chunk['pagina_pdf']} "
        f"NER: {ner_text} TEXTO: {chunk['texto']}"
    ).strip()

    emb = encode_text_mean_pool(texto_meta)
    texts.append(texto_meta)
    metadados.append({
        "titulo": chunk["titulo"],
        "autor": chunk["autor"],
        "pagina_pdf": chunk["pagina_pdf"],
        "chunk_id": chunk["chunk_id"],
        "ner": chunk.get("ner", [])
    })
    lista_emb.append(emb)

emb_matrix = np.vstack(lista_emb).astype("float32")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


### Armazenando meus vetores dentro do FAISS Index
Aqui armazeno meus vetores em um banco FAISS

In [49]:
embeddings = emb_matrix
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# estrutura tabular mínima 
df_subset = pd.DataFrame({
    "TextoManifestacao": texts,
    "titulo": [m["titulo"] for m in metadados],
    "autor": [m["autor"] for m in metadados],
    "pagina_pdf": [m["pagina_pdf"] for m in metadados],
    "chunk_id": [m["chunk_id"] for m in metadados],
    "ner": [m["ner"] for m in metadados],
    "embeddings": list(embeddings)
})

### Carregando meu LLM local

Carregamento de um LLM local para realização do RAG


In [50]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("TOKEN"),
)

def llm_generate(system_prompt: str, user_prompt: str, max_tokens: int = 256, temperature: float = 0.7):
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3.1-Terminus:novita",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=0.95,
    )
    return completion.choices[0].message.content

## Criando a Pipeline do RAG

- Aqui esta sendo feito um rewriting da query para gerar um texto mais objetivo para busca por semelhnaça de cosseno.

In [51]:
# remoçao do bloco pois estava dando incoerencia 

def requery(query):
    return query

### Realizando o reranking das top 10 textos, recuperando as 5 mais próximas

In [52]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

RERANK_ENTITY_BOOST = 0.25  # incremento multiplicativo para docs com interseção de entidades

tokenizer_rerank = AutoTokenizer.from_pretrained("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")
model_rerank = AutoModelForSequenceClassification.from_pretrained("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")
model_rerank.eval()

def _has_entity_intersection(query_entities, doc_ner_list):
    if not query_entities or not doc_ner_list:
        return False
    qset = set(e["text"].lower() for e in query_entities if e.get("text"))
    dset = set(e.get("text", "").lower() for e in doc_ner_list if isinstance(e, dict))
    return len(qset.intersection(dset)) > 0

def reranking(query, retrived_docs, top_k=10, query_entities=None, verbose=False):
    ents_text = " " + " ".join([e["text"] for e in (query_entities or [])]) if query_entities else ""
    query_with_ents = (query + ents_text).strip()

    pares = [(query_with_ents, doc) for _, doc in retrived_docs]
    if verbose:
        print("\nRealizando o re-ranking (Cross-Encoder) usando entidades + boost por interseção...\n")

    tokens = tokenizer_rerank(pares, padding=True, truncation=True, return_tensors="pt", max_length=800)

    with torch.no_grad():
        results = model_rerank(**tokens).logits.squeeze(-1)

    docs = [doc for _, doc in retrived_docs]
    idx_doc = [idx for idx, _ in retrived_docs]

    # bost nos scores se houver interseção de entidades
    boosted_scores = []
    for score, di in zip(results.tolist(), idx_doc):
        try:
            doc_ner = df_subset.iloc[di]["ner"]
        except Exception:
            doc_ner = []
        if _has_entity_intersection(query_entities or [], doc_ner):
            score = score * (1.0 + RERANK_ENTITY_BOOST)
        boosted_scores.append(score)

    doc_results = list(zip(idx_doc, docs, boosted_scores))
    top_docs = sorted(doc_results, key=lambda x: x[2], reverse=True)

    if verbose:
        print("Documentos Recuperados (com reforço de entidades + boost):")
        for i, (_, doc, valor) in enumerate(top_docs[:top_k]):
            print(f"{i+1} - Score: {valor:.4f} | Manifestação: {doc[:100]}...")

    return top_docs[:top_k]


### Realizando o Highligh dos trechos mais coerentes com as perguntas do usuario encontrados dentro dos top 5 textos

In [53]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter

HIGHLIGHT_CHUNK_SIZE = 500
HIGHLIGHT_CHUNK_OVERLAP = 120
HIGHLIGHT_TOP_PER_DOC = 2
HIGHLIGHT_GLOBAL_MAX_CHARS = 4000
HIGHLIGHT_MIN_INDEX_GAP = 3  # distância mínima entre índices de chunks

# utiliza a função de BERTimbau definida anteriormente
def encode_batch_mean_pool(texts):
    if not texts:
        return np.zeros((0, 768), dtype=np.float32)
    vecs = [encode_text_mean_pool(t) for t in texts]
    return np.vstack(vecs).astype("float32")


def highlight(query_emb, docs, lista_idx, verbose=False):

    if verbose:
        print("\nRealizando o highlight dos documentos (conservador + diversidade)...\n")
    dicionario_chunks = {}

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=HIGHLIGHT_CHUNK_SIZE,
        chunk_overlap=HIGHLIGHT_CHUNK_OVERLAP,
        separators=["\n\n", "\n", ". ", "? ", "! ", " "]
    )

    for idx, doc in zip(lista_idx, docs):
        chunks = text_splitter.split_text(doc)
        dicionario_chunks[idx] = chunks
        if verbose:
            print(f"Documento {idx}:")
            for j, chunk in enumerate(dicionario_chunks[idx][:2]):
                print(f"Chunk {j+1}: {chunk[:200]}...")
            print("\n")
    
    if verbose:
        print("\nSelecionando top chunks por similaridade (com diversidade)...\n")
    highlighted_chunks = []
    total_chars = 0

    for idx, chunks in dicionario_chunks.items():
        if not chunks:
            highlighted_chunks.append((idx, ""))
            continue
        embeddings_chunks = encode_batch_mean_pool(chunks)
        similarities = cosine_similarity(query_emb.reshape(1, -1), embeddings_chunks)
        sorted_indices = np.argsort(similarities[0])[::-1]

        selecionados = []
        selected_indices = []
        for i in sorted_indices:
            # respeita distância mínima entre índices
            if any(abs(i - si) < HIGHLIGHT_MIN_INDEX_GAP for si in selected_indices):
                continue
            c = chunks[i]
            if total_chars + len(c) <= HIGHLIGHT_GLOBAL_MAX_CHARS:
                selecionados.append(c)
                selected_indices.append(i)
            if len(selecionados) >= HIGHLIGHT_TOP_PER_DOC:
                break
        
        texto_final = "\n".join(selecionados)
        total_chars += sum(len(s) for s in selecionados)
        highlighted_chunks.append((idx, texto_final))
        if verbose:
            print(f"Documento {idx} selecionado com {len(selecionados)} chunks (indices={selected_indices}), total chars acumulados: {total_chars}")
            print("\n")
    
    return highlighted_chunks

### Função que opera o RAG

In [54]:
def rag_response(query, index, df_subset, top_k=10, verbose=False):
    # reconhecimento de entidades na query
    query_ents = extract_entities_pt(query)

    # expansão simples com entidades (sem LLM)
    busca = expand_query_with_entities(query, query_ents)

    # embedding da consulta expandida via BERTimbau
    query_embedding = encode_text_mean_pool(busca).astype("float32")

    # recuperação por similaridade de cosseno (contexts iniciais)
    distances, indices = index.search(np.array([query_embedding]), top_k)

    controle_idx = []
    retrieved_docs = []
    for i, idx in enumerate(indices[0]):
        retrieved_docs.append(df_subset.iloc[idx]['TextoManifestacao'])
        controle_idx.append(idx)

    # highlighting
    docs_highlight = highlight(query_emb=query_embedding, docs=retrieved_docs, lista_idx=controle_idx, verbose=verbose)

    # reranking com Cross-Encoder usando entidades
    docs_rerank = reranking(query=busca, retrived_docs=docs_highlight, top_k=top_k, query_entities=query_ents, verbose=verbose)

    # contexto para o LLM (top documentos)
    context = "\n\n".join([doc for _, doc, _ in docs_rerank])

    system = (
        "Você é um grande assistente de IA expecialista em Sóciologia e Filosófia clássica que responde perguntas baseado no contexto fornecido."
        "Monte respostas coesas em completas utilizando o material fornecido como base, e não se esqueca de mencionar o trecho utilizado como referencia ao final da resposta, atrelando ele ao que foi respondido."
        "Se não houver informação suficiente, diga: 'Eu não tenho informações suficientes para responder esta pergunta.'"
    )
    user = f"Contexto:\n{context}\n\nPergunta:\n{query}"

    response = llm_generate(system_prompt=system, user_prompt=user, max_tokens=256, temperature=0.7)

    # preparar saídas estruturadas para avaliação
    initial_contexts = [
        {
            "idx": int(iidx),
            "preview": df_subset.iloc[iidx]['TextoManifestacao'][:300],
            "titulo": df_subset.iloc[iidx]['titulo'],
            "autor": df_subset.iloc[iidx]['autor'],
            "pagina_pdf": df_subset.iloc[iidx]['pagina_pdf'],
            "chunk_id": df_subset.iloc[iidx]['chunk_id'],
            "ner": df_subset.iloc[iidx]['ner'],
        }
        for iidx in controle_idx
    ]

    reranked = [
        {
            "idx": int(iidx),
            "score": float(score),
            "preview": doc[:300],
            "titulo": df_subset.iloc[iidx]['titulo'],
            "autor": df_subset.iloc[iidx]['autor'],
            "pagina_pdf": df_subset.iloc[iidx]['pagina_pdf'],
            "chunk_id": df_subset.iloc[iidx]['chunk_id'],
            "ner": df_subset.iloc[iidx]['ner'],
        }
        for (iidx, doc, score) in docs_rerank
    ]

    return {
        "question": query,
        "query_entities": query_ents,
        "requery_expanded": busca,
        "initial_indices": [int(i) for i in controle_idx],
        "initial_contexts": initial_contexts,
        "reranked": reranked,
        "answer": response,
    }

### Testando o sistema de RAG

In [55]:
test_questions = [
    "O que Sócrates queria dizer com: Mas, cidadãos atenienses, parece -me que também os artífices tinham o mesmo defeito dos poetas?",
    "O que epicuro quer dizer em sua carta para meneceu  sobre a morte, onde aponta à idéia de que a morte para nós não é nada e não significa nada para nós?",
    "O que Sócrates diz no trecho onde explica o que aconteceria se os homens fossem libertados de suas correntes e curados de sua desrazão?"
]

for question in test_questions:
    result = rag_response(
        query=question,
        index=index,
        df_subset=df_subset,
        top_k=10,
        verbose=False
    )

    print("\n=== Avaliação RAG ===")
    print(f"Pergunta: {result['question']}")
    ents = ", ".join(sorted(set(e.get('text','') for e in result.get('query_entities', []) if e.get('text'))))
    print(f"Entidades (query): {ents if ents else '—'}")
    print(f"Query Expandida: {result['requery_expanded']}")

    print("\nContextos iniciais (similaridade):")
    for i, ctx in enumerate(result['initial_contexts'][:5], start=1):
        meta = f"[idx={ctx['idx']}, pag={ctx['pagina_pdf']}, autor={ctx['autor']}, título={ctx['titulo']}]"
        print(f"{i}. {meta} | {ctx['preview'][:200]}...")

    print("\nContextos após reranking (Cross-Encoder + boost):")
    for i, ctx in enumerate(result['reranked'][:5], start=1):
        meta = f"[idx={ctx['idx']}, score={ctx['score']:.4f}, pag={ctx['pagina_pdf']}, autor={ctx['autor']}, título={ctx['titulo']}]"
        print(f"{i}. {meta} | {ctx['preview'][:200]}...")

    print("\nResposta do LLM:")
    print(result['answer'])
    print("====================\n")


=== Avaliação RAG ===
Pergunta: O que Sócrates queria dizer com: Mas, cidadãos atenienses, parece -me que também os artífices tinham o mesmo defeito dos poetas?
Entidades (query): Sócrates
Query Expandida: O que Sócrates queria dizer com: Mas, cidadãos atenienses, parece -me que também os artífices tinham o mesmo defeito dos poetas? Sócrates

Contextos iniciais (similaridade):
1. [idx=18, pag=6, autor=Platão, título=Apologia de Sócrates] | TITULO: Apologia de Sócrates AUTOR: Platão PAGINA: 6 NER:  TEXTO: VI 
Depois prossegui se mais me deter. embora vendo, amargurado e 
temeroso, que estava incorrendo em ódio; mas também me parecia 
dev...
2. [idx=10, pag=4, autor=Platão, título=Apologia de Sócrates] | TITULO: Apologia de Sócrates AUTOR: Platão PAGINA: 4 NER: Paros Cálias de Hipônico Cálias TEXTO: persuadir a e star com eles, deixando as outras conversações, 
compensado-os com dinheiro e proporciona...
3. [idx=20, pag=6, autor=Platão, título=Apologia de Sócrates] | TITULO: Apologia de